# <center> Running remotely using Pyro4
### <center> In this demo you will learn how to run the low-level software on the QICK as a server, and "proxy" it over the network to a separate client computer that runs the high-level software.

Pyro4 is a software package that lets you "proxy" a Python object so it can be accessed from another computer.

https://pyro4.readthedocs.io/en/stable/intro.html

we can use Pyro4 to proxy the QickSoc object. This allows you to take any of the demo notebooks and run it on a different computer, and all you need to do is replace the `QickSoc()` initialization with some Pyro4 initialization.

## Pros and cons
There are advantages to using Pyro4:
* The QickSoc doesn't get reinitialized every time you restart your notebook. If your experiment cares about the readout phase, which gets reset and must be recalibrated every time the FPGA gets reprogrammed, this is a big deal.
* You can write and run your QickProgram (or AveragerProgram/RAveragerProgram) on any computer. If you're used to using a graphical IDE, or you want to run your programs in an environment other than what's available on the QICK (i.e. Jupyter notebooks and command-line scripts over SSH), this is a big deal. Your PC is probably also more responsive than the QICK (which has approximately the CPU/RAM/storage performance of a Raspberry Pi).

But there are disadvantages that you should be aware of:
* There's some overhead: you need to start a nameserver and server in addition to what you would normally do.
* Debugging is harder, because exceptions on the server don't get printed on the client (you just get a generic Pyro4 error) without some extra work: https://pyro4.readthedocs.io/en/stable/errors.html
* Any messages printed by QickSoc code only show up on the server.
* Pyro4 only proxies methods and specially registered properties of the QickSoc. This means that you can't access all the members of the QickSoc (e.g. the readout and generator objects). The demos are written to avoid Pyro4-incompatible operations, but you may have code that needs to be rewritten.
* It's easy to lose track of which version of the QICK library you have installed in which place.

## Overview, requirements
There are three components to a Pyro4 setup: the nameserver, the server (also known as the daemon), and the client. Start them in this order.

The server must run on the QICK, but the nameserver and client can be anywhere. To keep things self-contained, this demo is structured to put everything on the QICK, but we note what you would do differently if running the nameserver and/or client elsewhere.

The nameserver, server, and client must all be on the same network - a lab LAN is fine. Tailscale (http://tailscale.com/) works well in the case where everything has Internet access, but not everything is on the same LAN.

You need to install pyro4, both on the QICK and the computer(s) running the nameserver and client - it's a Python module, available through pip (`sudo pip3 install Pyro4`) or apt (`sudo apt install python3-pyro4`).

The nameserver and server run continuously, so if you want to run them in Jupyter notebooks you need to use separate notebooks for the nameserver, server, and client. Alternatively, you could run the nameserver and server as command-line scripts (also provided here) which you would run over SSH, perhaps using a tool such as `screen` which provides persistent terminal sessions that survive loss of an SSH connection.

## Nameserver
The nameserver is how the client finds the server. You should only run one nameserver on a network.

##### Hostname
The hostname defines how clients and servers can connect to the nameserver. Specify the hostname or IP address that both the client and server will use to connect to the nameserver (if you're running everything on the QICK you could specify "localhost" or "127.0.0.1" to restrict access to programs running locally). "0.0.0.0" means that the nameserver listens on all available network interfaces, which is usually what you want (but might not work in all cases?).

##### Port
Because the default nameserver port (9090) conflicts with the port PYNQ uses for the Jupyter notebook server, we use port 8888 in this demo. If you put the nameserver elsewhere, you might prefer to use the default port and save a few lines of code. In this case you would not define the ns_port variable and omit that parameter in the `pyro4-ns` and `locateNS()` commands.

### Running the nameserver
To run the nameserver, run the following shell command (either in an SSH session or a separate notebook):
```
PYRO_SERIALIZERS_ACCEPTED=pickle PYRO_PICKLE_PROTOCOL_VERSION=4 pyro4-ns -n 0.0.0.0 -p 8888
```

There's also a script in this directory, which takes the same options (again, this could run over SSH or in a notebook):
```
./nameserver.sh -n 0.0.0.0 -p 8888
```

Typical output looks like this:

```
xilinx@pynq216:~$ export PYRO_SERIALIZERS_ACCEPTED=pickle PYRO_PICKLE_PROTOCOL_VERSION=4 pyro4-ns -n 0.0.0.0 -p 8888
Broadcast server running on 0.0.0.0:9091
NS running on 0.0.0.0:8888 (0.0.0.0)
Warning: HMAC key not set. Anyone can connect to this server!
URI = PYRO:Pyro.NameServer@0.0.0.0:8888
```

The nameserver doesn't print anything when servers and clients connect to it - don't worry! Leave it running.

## Server
The server runs the QickSoc code and exposes it to the network. 

You can choose whatever server name you want, as long as you use the same one on the client. You could run multiple QICKs, each with a Pyro4 server, using different server names.

You can use the notebook cell below (copy it into another notebook so you can leave it running), or the script in this directory (use `-h` for help), which must run as root:

```
sudo ./server.py myqick -n localhost -p 8888
```

In [ ]:
import Pyro4
from qick import QickSoc

ns_host = "localhost"
ns_port = 8888
server_name = "myqick"

print("looking for nameserver . . .")
Pyro4.config.REQUIRE_EXPOSE = False
Pyro4.config.SERIALIZER = "pickle"
Pyro4.config.SERIALIZERS_ACCEPTED=set(['pickle'])
Pyro4.config.PICKLE_PROTOCOL_VERSION=4
ns = Pyro4.locateNS(host=ns_host, port=ns_port)
print("found nameserver")

# if we have multiple network interfaces, we want to register the daemon using the IP address that faces the nameserver
host = Pyro4.socketutil.getInterfaceAddress(ns._pyroUri.host)
daemon = Pyro4.Daemon(host=host)

# if you want to use a different firmware image or set some initialization options, you would do that here
soc = QickSoc()
print("initialized QICK")

# register the QickSoc in the daemon (so the daemon exposes the QickSoc over Pyro4)
# and in the nameserver (so the client can find the QickSoc)
ns.register(server_name, daemon.register(soc))
print("registered QICK")

# register in the daemon all the objects we expose as properties of the QickSoc
# we don't register them in the nameserver, since they are only meant to be accessed through the QickSoc proxy
# https://pyro4.readthedocs.io/en/stable/servercode.html#autoproxying
# https://github.com/irmen/Pyro4/blob/master/examples/autoproxy/server.py
for obj in soc.autoproxy:
    daemon.register(obj)
    print("registered member "+str(obj))
    
print("starting daemon")
daemon.requestLoop() # this will run forever until interrupted

## Client
The client controls the experiment. The client doesn't need the PYNQ device drivers, so it can be a Linux or Windows PC (you should be able to install the QICK libraries as usual, the PYNQ library will be skipped on systems that don't support it).

The cells below contain the initialization code that replace the typical initialization steps of:
```
from qick import QickSoc
soc = QickSoc()
soccfg = soc
```

In [3]:
import Pyro4
from qick import QickConfig
Pyro4.config.SERIALIZER = "pickle"
Pyro4.config.PICKLE_PROTOCOL_VERSION=4

ns_host = "localhost"
ns_port = 8888
server_name = "myqick"

ns = Pyro4.locateNS(host=ns_host, port=ns_port)

Let's see what is registered on the nameserver:

In [4]:
# print the nameserver entries: you should see the QickSoc proxy
for k,v in ns.list().items():
    print(k,v)

Pyro.NameServer PYRO:Pyro.NameServer@0.0.0.0:8888
myqick PYRO:obj_f90d361dab334faa8318e41b5810cb04@127.0.0.1:44659


Now, connect to the server. The Proxy object replaces the QickSoc in your code. There's one additional step: you need to create a QickConfig object that contains all the information necessary to compile programs.

In the demo notebooks both `soc` and `soccfg` point to the same QickSoc object; when using Pyro, `soc` is a Proxy object and `soccfg` is a QickConfig object.

In [5]:
soc = Pyro4.Proxy(ns.lookup(server_name))
soccfg = QickConfig(soc.get_cfg())
print(soccfg)


QICK configuration:

	Board: ZCU216

	Global clocks (MHz): tProcessor 430.080, RF reference 245.760

	7 signal generator channels:
	0:	axis_signal_gen_v4 - tProc output 1, switch ch 0, maxlen 65536
		DAC tile 2, ch 0, 32-bit DDS, fabric=430.080 MHz, fs=6881.280 MHz
	1:	axis_signal_gen_v4 - tProc output 2, switch ch 1, maxlen 65536
		DAC tile 2, ch 1, 32-bit DDS, fabric=430.080 MHz, fs=6881.280 MHz
	2:	axis_signal_gen_v4 - tProc output 3, switch ch 2, maxlen 65536
		DAC tile 2, ch 2, 32-bit DDS, fabric=430.080 MHz, fs=6881.280 MHz
	3:	axis_signal_gen_v4 - tProc output 4, switch ch 3, maxlen 65536
		DAC tile 2, ch 3, 32-bit DDS, fabric=430.080 MHz, fs=6881.280 MHz
	4:	axis_signal_gen_v4 - tProc output 5, switch ch 4, maxlen 65536
		DAC tile 3, ch 0, 32-bit DDS, fabric=430.080 MHz, fs=6881.280 MHz
	5:	axis_signal_gen_v4 - tProc output 6, switch ch 5, maxlen 65536
		DAC tile 3, ch 1, 32-bit DDS, fabric=430.080 MHz, fs=6881.280 MHz
	6:	axis_signal_gen_v4 - tProc output 7, switch ch 6, maxl